In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from scipy.io import loadmat
import cv2
import skimage as io
import PIL
from PIL import Image
import os
from google.colab.patches import cv2_imshow

In [ ]:
def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

In [ ]:
class BackgroundModel(nn.Module):

	def __init__(self, input_size, hidden_size, num_layers):
		super(BackgroundModel, self).__init__()
		self.rnn = nn.GRU(input_size, hidden_size, num_layers)
		self.linear = nn.Linear(hidden_size, input_size)

	def forward(self, x_in):
		x_in_shape = x_in
		x_input1 = x_in[0].unsqueeze(0)
		latent, _ = self.rnn(x_input1)
		bg_es = self.linear(latent).view(x_in_shape.size())
		loss = RMSELoss(bg_es, x_input1)
		return bg_es, loss


In [ ]:
# defining the video file path and check input shape:

video_file = 'Replace the path of video'
cap = cv2.VideoCapture(video_file)
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
x = torch.FloatTensor(gray).unsqueeze(0).unsqueeze(0).view(1, 1, -1)
x.shape

In [ ]:
path_of_org = 'Replace the path you want save the original frames'
path_of_bg = 'Replace the path you want save the background frames'
path_of_fg = 'Replace the path you want save the foreground frames'


model = BackgroundModel(input_size=307200, hidden_size=40, num_layers=2)
optimizer = torch.optim.Rprop(model.parameters(), lr=0.02)
bg = torch.FloatTensor(np.zeros([1,307200]))


print('loading data..')

#open the video file
cap = cv2.VideoCapture(video_file)


print('start..')
#read the first frame
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
x = torch.FloatTensor(gray).unsqueeze(0).unsqueeze(0).view(1, 1, -1)
i = 1

# loop over the frames
while ret:
	print('Frame: ', i)

	#pre-processing the frame
	x = torch.FloatTensor(gray).unsqueeze(0).unsqueeze(0).view(1, 1, -1)

	#passing frame through model
	bg_es, loss = model.forward(x)

  #backpropagation
	optimizer.zero_grad()
	loss.backward()
	optimizer.step()

	ret, frame = cap.read()
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	img_org = x.reshape(480, 640)
	img_org = np.array(img_org, dtype = np.uint8)
	cv2.imwrite(path_of_org + str(i) + '.jpg', img_org)

	bg_result = bg_es.squeeze(dim=1)
	img_bg = bg_result.reshape(480, 640)
	img_bg = img_bg.detach().numpy()
	img_bg = np.array(img_bg, dtype = np.uint8)
	cv2.imwrite(path_of_bg + str(i) + '.jpg', img_bg)

	img_fg = cv2.subtract(img_org, img_bg)
	cv2.imwrite(path_of_fg + str(i) + '.jpg', img_fg)


	img_result = np.concatenate((img_org, img_bg, img_fg), axis=1)
	cv2_imshow(img_result)

	i = i+1
#releasing video file
cap.release()

